In [14]:
import numpy as np
from scipy.sparse import csr_matrix

In [15]:
import re
t_path = "../../pyths/data/nba_1.txt"
m_space = re.compile(r'\s+')
point = re.compile(r'\W+')

In [16]:
def read_text(t_path):
    whole_text = []
    whole_entity = {}
    vocab = set()
    m_space = re.compile(r'\s+')
    point = re.compile(r'\W+')
    with open(t_path, 'r', encoding='utf8') as file:
        for line in file:
            line = line.strip().replace(u'\xa0', u' ').replace(u"\'", u' ').replace(u'\.', u' ')
            text, entity = line.lower().split('\t')
            text = point.sub(' ', text)
            for token in entity.split(','):
                index = whole_entity.setdefault(token, len(whole_entity))
            t_l = []
            whole_text.append(t_l)
            for token in text.strip('\t').split(' '):
                if token not in whole_entity:
                    vocab.add(token)
                    t_l.append(token)
    indptr = [0]
    indice = []
    data = []
    vocab = {}
    for d in whole_text:
        #print(d)
        for token in d:
            index = vocab.setdefault(token, len(vocab))
            #print(token, index)
            indice.append(index)        
            data.append(1)
        indptr.append(len(indice))

    bow = csr_matrix((data, indice, indptr), dtype=int).toarray()
    
    return whole_text, whole_entity, bow, vocab


In [17]:
whole_text, whole_entity, bow, vocab = read_text(t_path)

In [44]:
# configuration param
T_marker = 1 # time period가 10까지 간다고 가정
N_story = 5 # storyline은 5까지 간다
N_topic = 8
N_docs = len(whole_text)
N_Voca = len(vocab)
N_NER = len(whole_entity)
N_epoch = 3 # 앞선 3개 epoch의 정보만을 참조한다
N_token = np.zeros([N_docs], dtype='uint32')

In [49]:
# evolutionary matrix
# storyline s and topic z
sig_sz = np.zeros([N_story, N_topic, N_Voca, N_epoch])
tau_s = np.zeros([N_story, N_topic, N_epoch])
v_s = np.zeros([N_story, N_NER, N_epoch])
mu_m = np.zeros([N_epoch])
for m in range(N_epoch):
    mu_m[m] = np.exp(-0.5*m)
 
 
pi = np.zeros([T_marker, N_story])
theta = np.zeros([T_marker, N_story, N_topic])
omega = np.zeros([T_marker, N_story, N_NER])
phi = np.zeros([T_marker, N_story, N_topic, N_Voca])
s_t = np.zeros([N_story])
e = np.zeros([T_marker, N_docs, N_NER])
 
 
# hyperparameter
gamma = np.array([0.1] * N_story)
alpha = np.ones([N_story, N_topic]) * 0.1
eps = np.array([0.01] * N_NER)
beta = np.ones([N_story, N_topic, N_Voca]) * 0.01
 
n_j = np.zeros([N_docs, N_story], dtype='uint32')
n_jk = np.zeros([N_docs, N_story, N_topic], dtype='uint32')
n_jkv = np.zeros([N_docs, N_story, N_topic, N_Voca], dtype='uint32')
s_indicator = np.zeros([N_docs])
s_t = np.zeros([N_story])

In [50]:
# initialize latent variable
z_agg = []
w_agg = []
std_agg_by_d = []
for d_id, doc in enumerate(whole_text): #print(np.random.multinomial(1, pi[time_tick, :]).argmax())        
    N_token[d_id] = len(doc) # 문서 당 길이
    z_d = np.zeros([N_token[d_id]], dtype=np.uint32)
    w_d = np.zeros([N_token[d_id]], dtype=np.uint32)
    #s_indicator[d_id] = np.random.multinomial(1, pi[t]).argmax()
    s_indicator[d_id] = np.random.randint(N_story)
    #print(int(s_indicator[d_id]))
    std_agg = []
    for w_pos, token in enumerate(doc):
        z_d[w_pos] = np.random.randint(N_topic)
        w_d[w_pos] = vocab[token]
        n_jk[d_id, int(s_indicator[d_id]), int(z_d[w_pos])] += 1
        n_jkv[d_id, int(s_indicator[d_id]), int(z_d[w_pos]), int(w_d[w_pos])] += 1
        story_topic_word_d = (s_indicator[d_id], z_d[w_pos], w_d[w_pos])
        std_agg.append(story_topic_word_d)
    z_agg.append(z_d)
    w_agg.append(w_d)
    std_agg_by_d.append(std_agg)
    n_j[d_id, int(s_indicator[d_id])] += N_token[d_id]

for s_id in s_indicator:
    s_t[int(s_id)] += 1

In [65]:
first_term = np.zeros([N_docs, N_story])
 
alpha = np.ones([N_story, N_topic]) * 0.1
beta = np.ones([N_story, N_topic, N_Voca]) * 0.01
nume_3 = np.ones([N_docs, N_story, N_topic], dtype='float64')
denom_3 = np.ones([N_docs, N_story], dtype='float64')
nume_4 = np.ones([N_docs, N_story, N_topic, N_Voca], dtype='float64')
denom_4 = np.ones([N_docs, N_story, N_topic], dtype='float64')
 
third_term = np.ones([N_docs, N_story, N_topic], dtype='float')
log_third_term = np.ones([N_docs, N_story], dtype='float64')
 
fourth_term = np.ones([N_docs, N_story, N_topic, N_Voca], dtype='float64')
fourth_term_prod = np.ones([N_docs, N_story, N_topic], dtype='float64')
 
N_j = np.zeros([N_docs, N_story], dtype='uint32')

In [66]:
for d, doc in enumerate(whole_text):
    for s in range(N_story):
        N_j[d, s] = s_t[s] - 1 if s_indicator[d] == s else s_t[s]
    
    first_term = (N_j + gamma) / ((N_docs - 1) + N_story * gamma)
    s_indicator[d] = np.random.multinomial(1, first_term[d]).argmax()
    ## 여기까지 함
    np.sum(n_j, 0) - n_j[d] + np.sum(alpha, 1)

In [67]:
s_indicator

array([ 3.,  2.,  0.,  3.,  0.,  2.,  0.,  3.,  0.,  0.,  1.,  3.,  4.,
        1.,  0.,  3.,  1.])

In [68]:
n_j[d, int(s_indicator[d])]

0

In [ ]:
alpha = np.ones([N_story, N_topic]) * 0.1
beta = np.ones([N_story, N_topic, N_Voca]) * 0.01
nume_3 = np.ones([N_docs, N_story, N_topic], dtype='float64')
denom_3 = np.ones([N_docs, N_story], dtype='float64')
nume_4 = np.ones([N_docs, N_story, N_topic, N_Voca], dtype='float64')
denom_4 = np.ones([N_docs, N_story, N_topic], dtype='float64')
 
third_term = np.ones([N_docs, N_story, N_topic], dtype='float')
log_third_term = np.ones([N_docs, N_story], dtype='float64')
 
fourth_term = np.ones([N_docs, N_story, N_topic, N_Voca], dtype='float64')
fourth_term_prod = np.ones([N_docs, N_story, N_topic], dtype='float64')
 
N_j = np.zeros([N_docs, N_story], dtype='uint32')
 
for d, doc in enumerate(whole_text):
    denom_3[d, :] *= np.sum(n_j, 0) - n_j[d] + np.sum(alpha, 1)
    nume_3[d] *= np.sum(n_jk, 0) - n_jk[d] + alpha
 
    
    for s in range(N_story):
        N_j[d, s] = s_t[s] - 1 if s_indicator[d] == s else s_t[s]
        
    
        denom_4[d] *= np.sum(n_jk, 0) - n_jk[d] + np.sum(beta[0][0])
        
    #print(denom_4[d])
    
    for t in range(N_topic):
        for v in range(N_Voca):
        
            fourth_term[d, :, t, v] = nume_4[d, :, t, v]/denom_4[d, :, t]
        fourth_term_prod[d] = np.sum(np.log10(fourth_term[d]), 2)
    
    for t in range(N_topic):
        third_term[d, :, t] = nume_3[d, :, t] / denom_3[d] 
        print(fourth_term_prod[d, :, t], third_term[d, :, t])
        
    log_third_term[d] =np.sum(np.log10(third_term[d]+fourth_term_prod[d]), 1)  
    
    
    first_term = (N_j + gamma) / ((N_docs - 1) + N_story * gamma)


In [ ]:
for t in range(T_marker):
    pi[t] = np.random.dirichlet(gamma) # 시점 T에서 문서에 할당할 Story indicator의 분포
    for s in range(N_story):
        theta[t, s] = np.random.dirichlet(alpha[s, :])
        omega[t, s] = np.random.dirichlet(eps)
        for z in range(N_topic):
            phi[t, s, z] = np.random.dirichlet(beta[0, 0, :])
    
    z_agg = []
    w_agg = []
    std_agg_by_d = []
    for d_id, doc in enumerate(whole_text): #print(np.random.multinomial(1, pi[time_tick, :]).argmax())        
        N_token[d_id] = len(doc) # 문서 당 길이
        z_d = np.zeros([N_token[d_id]], dtype=np.uint32)
        w_d = np.zeros([N_token[d_id]], dtype=np.uint32)
        #s_indicator[d_id] = np.random.multinomial(1, pi[t]).argmax()
        s_indicator[d_id] = np.random.randint(N_story)
        #print(int(s_indicator[d_id]))
        std_agg = []
        for w_pos, token in enumerate(doc):
#            z_d[w_pos] = np.random.multinomial(1, theta[t, int(s_indicator[d_id]), :]).argmax()
            z_d[w_pos] = np.random.randint(N_topic)
            w_d[w_pos] = vocab[token]
            #print(z_d[w_pos], w_d[w_pos])
            n_jk[d_id, int(s_indicator[d_id]), int(z_d[w_pos])] += 1
            n_jkv[d_id, int(s_indicator[d_id]), int(z_d[w_pos]), int(w_d[w_pos])] += 1
            story_topic_word_d = (s_indicator[d_id], z_d[w_pos], w_d[w_pos])
            std_agg.append(story_topic_word_d)
        z_agg.append(z_d)
        w_agg.append(w_d)
        std_agg_by_d.append(std_agg)
        n_j[d_id, int(s_indicator[d_id])] += N_token[d_id]
        
        
    for s_id in s_indicator:
        s_t[int(s_id)] += 1

In [ ]:
def init(t=1):
    pi[t] = np.random.dirichlet(gamma) # 시점 T에서 문서에 할당할 Story indicator의 분포
    for s in range(N_story):
        theta[t, s] = np.random.dirichlet(alpha[0, :])
        omega[t, s] = np.random.dirichlet(eps)
        for z in range(N_topic):
            phi[t, s, z] = np.random.dirichlet(beta[0, 0, :])

    z_agg = []
    w_agg = []
    std_agg_by_d = []
    for d_id, doc in enumerate(whole_text): #print(np.random.multinomial(1, pi[time_tick, :]).argmax())        
        N_token[d_id] = len(doc) # 문서 당 길이
        z_d = np.zeros([N_token[d_id]], dtype=np.uint32)
        w_d = np.zeros([N_token[d_id]], dtype=np.uint32)
        s_indicator[d_id] = int(np.random.multinomial(1, pi[t]).argmax())
        std_agg = []
        for w_pos, token in enumerate(doc):
            z_d[w_pos] = np.random.multinomial(1, theta[t, s, :]).argmax()
            w_d[w_pos] = vocab[token]
            print(z_d[w_pos], w_d[w_pos])
            n_jk[d_id, int(s_indicator[d_id]), int(z_d[w_pos])] += 1
            n_jkv[d_id, int(s_indicator[d_id]), int(z_d[w_pos]), int(w_d[w_pos])] += 1
            story_topic_word_d = (s_indicator[d_id], z_d[w_pos], w_d[w_pos])
            std_agg.append(story_topic_word_d)
        z_agg.append(z_d)
        w_agg.append(w_d)
        std_agg_by_d.append(std_agg)
        n_j[d_id, int(s_indicator[d_id])] += N_token[d_id]


    for s_id in s_indicator:
        s_t[int(s_id)] += 1